In [33]:
import MetaTrader5 as mt5
import pandas as pd 
mt5.initialize()

True

In [34]:
def RolarTravaAltaPut(ativosLista, comprei, vendi ):
    # [travaFeitaC (comprada) , travaFeitaV (vendida), travaNovaC, travaNovaV]
    travaFeitaC = mt5.symbol_info(ativosLista[0])
    travaFeitaV = mt5.symbol_info(ativosLista[1])
    travaNovaC  = mt5.symbol_info(ativosLista[2])
    travaNovaV  = mt5.symbol_info(ativosLista[3])
    ativoBase = travaFeitaC.basis
    ativoBaseInfo = mt5.symbol_info(ativoBase)

    lin,col = (3,9)
    matriz = [[0 for i in range(col)] for j in range(lin)]
    ###########################################
    # coluna 0 - ativos
    matriz[0][0] = travaFeitaC.name 
    matriz[1][0] = travaFeitaV.name
    matriz[2][0] = 'Ʃ'

    # coluna 1 - strike feitos
    matriz[0][1] = travaFeitaC.option_strike
    matriz[1][1] = travaFeitaV.option_strike
    matriz[2][1] = matriz[1][1] - matriz[0][1]

    # coluna 2 - valores da trava feita
    matriz[0][2] = round(comprei,2)
    matriz[1][2] = round(vendi,2)
    matriz[2][2] = round(matriz[1][2] + matriz[0][2], 2)

    # coluna 3 - valores para desmontar a trava feita
    matriz[0][3] = round(travaFeitaC.bid,2)
    matriz[1][3] = round(-travaFeitaV.ask,2)
    matriz[2][3] = round(matriz[1][3] + matriz[0][3], 2)

    # coluna 4 - ativos novos
    matriz[0][4] = travaNovaC.name 
    matriz[1][4] = travaNovaV.name
    matriz[2][4] = 'Ʃ'

    
    # coluna 5 - strikes novos
    matriz[0][5] = round(travaNovaC.option_strike,2)
    matriz[1][5] = round(travaNovaV.option_strike,2)
    matriz[2][5] = round(matriz[1][1] - matriz[0][1], 2)

    # coluna 6 - valores para montar nova trava
    matriz[0][6] = -travaNovaC.ask
    matriz[1][6] = travaNovaV.bid
    if matriz[1][6] == 0:
        fff = input('Entre o valor do Bid ' + travaNovaV.name + ': ')
        matriz[1][6] = float(fff)
    matriz[2][6] = matriz[1][6] + matriz[0][6]

    # coluna 7 - ..
    matriz[0][7] = 'Ativo Base'
    matriz[1][7] = 'last'
    matriz[2][7] = 'Resultado final'
  

    # coluna 8 - Basis
    matriz[0][8] = ativoBase
    matriz[1][8] = ativoBaseInfo.last
    matriz[2][8] = (matriz[2][2] + matriz[2][3] + matriz[2][6])
   

    df = pd.DataFrame(matriz, columns = ['Ativo', 'Strike', '$ Montei', '$ Desmontar' ,'Ativo Novo','Strike Novo', '$ montar novo' , '...', 'Basis'])
    print(df)
    nome_arq = 'RolarTravaAltaPut_' + ativoBase  + '.xlsx'
    df.to_excel(nome_arq)


In [38]:
RolarTravaAltaPut(['BBSES220', 'BBSES235', 'BBSET220', 'BBSET235'], -0.15, 0.4)

      Ativo  Strike  $ Montei  $ Desmontar Ativo Novo  Strike Novo  \
0  BBSES220    22.0     -0.15         0.10   BBSET220         22.0   
1  BBSES235    23.5      0.40        -0.97   BBSET235         23.5   
2         Ʃ     1.5      0.25        -0.87          Ʃ          1.5   

   $ montar novo              ...  Basis  
0          -0.41       Ativo Base  BBSE3  
1           1.06             last  22.69  
2           0.65  Resultado final   0.03  


In [37]:
RolarTravaAltaPut(['B3SAS805', 'B3SAS17',  'B3SAT148', 'B3SAT170'], -0.1, 0.66)

      Ativo  Strike  $ Montei  $ Desmontar Ativo Novo  Strike Novo  \
0  B3SAS805   14.66     -0.10         0.04   B3SAT148        14.59   
1   B3SAS17   16.66      0.66        -0.87   B3SAT170        16.51   
2         Ʃ    2.00      0.56        -0.83          Ʃ         2.00   

   $ montar novo              ...  Basis  
0          -0.25       Ativo Base  B3SA3  
1           1.00             last  15.85  
2           0.75  Resultado final   0.48  
